# Пайплайн Baseline подхода с GigaChat

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_gigachat import Gigachat

load_dotenv(find_dotenv())

def init_gigachat():
    return Gigachat(credentials="ваш_ключ_авторизации", model="GigaChat-Max", verify_ssl_certs=False, temperature=1e-15, timeout=100)

llm = init_gigachat()

In [ ]:
question = "Как установить PhotoShop?"
response = llm.invoke(question)

# Пайплайн подхода LLaVA

In [ ]:
from transformers import BitsAndBytesConfig, LlavaNextProcessor, LlavaNextForConditionalGeneration
from PIL import Image
import io
import pandas as pd
from typing import Tuple
from rag_env import INPUT_DATA


quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    low_cpu_mem_usage=True,
    use_flash_attention_2=True
)

def format_output(raw_output, processor: LlavaNextProcessor, prompt: str) -> str:
    out = processor.decode(raw_output[0], skip_special_tokens=True)
    out_prompt = prompt.replace("<image>", " ").strip()
    formatted_output = out.replace(out_prompt, "").strip()
    return formatted_output


def llava_call(prompt: str, model: LlavaNextForConditionalGeneration, processor: LlavaNextProcessor, device: str, image: Image=None) -> str:

    inputs = processor(prompt, image, return_tensors="pt").to(device)
    raw_output = model.generate(**inputs, max_new_tokens=300)
    formatted_output = format_output(raw_output, processor, prompt)
    return formatted_output


def load_llava_model(model_id: str) -> Tuple[LlavaNextForConditionalGeneration, LlavaNextProcessor]:
    processor = LlavaNextProcessor.from_pretrained(model_id)
    # uncomment to use quantized version of the model
    # model = LlavaNextForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")
    model = LlavaNextForConditionalGeneration.from_pretrained(model_id, device_map="auto")
    return model, processor


In [ ]:
device = "cuda"
model, processor = load_llava_model("llava-hf/llava-v1.6-mistral-7b-hf")
model = model.eval()


prompt = f"[INST]\nQuestion:\n{question}\n\n[/INST]"
question = "What are the possible positions of the manual operator and what colors are associated with each position?"

llava_response_no_img = llava_call(no_img_prompt, model, processor, device)
print(llava_response_no_img)

# Оценка ответа

In [ ]:
def init_deepseek():
    return Gigachat(model="DeepSeek-R1", verify_ssl_certs=False, temperature=1e-15, timeout=100)

llm = init_deepseek()

In [ ]:
from langchain.prompts import ChatPromptTemplate
from prompts import ANSWER_CORRECTNESS_SYSTEM, ANSWER_CORRECTNESS_USER

answer_correctness = ChatPromptTemplate.from_messages(
    [
        ("system", ANSWER_CORRECTNESS_SYSTEM),
        ("human", ANSWER_CORRECTNESS_USER)
    ]
)
llm = init_deepseek()

answer_correctness_chain = answer_correctness | llm
response = answer_correctness_chain.invoke("question": question, "reference_answer": reference_answer, "generated_answer": generated_answer)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from prompts import ANSWER_RELEVANCE_SYSTEM, ANSWER_RELEVANCE_USER

answer_correctness = ChatPromptTemplate.from_messages(
    [
        ("system", ANSWER_CORRECTNESS_SYSTEM),
        ("human", ANSWER_CORRECTNESS_USER)
    ]
)
llm = init_deepseek()

answer_correctness_chain = answer_correctness | llm
response = answer_correctness_chain.invoke("query": question, "text": generated_answer)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from prompts import CONTEXT_RELEVANCE_TEXT_SYSTEM, CONTEXT_RELEVANCE_TEXT_USER

answer_correctness = ChatPromptTemplate.from_messages(
    [
        ("system", ANSWER_CORRECTNESS_SYSTEM),
        ("human", ANSWER_CORRECTNESS_USER)
    ]
)
llm = init_deepseek()

answer_correctness_chain = answer_correctness | llm
response = answer_correctness_chain.invoke("query": question, "context": generated_answer)